In [26]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import h5py
from pathlib import Path
import os
import sys
sys.path.append(os.getcwd() + '/../..')
sys.path.append(os.getcwd() + '/..')
from filepaths import filepath_from_ml_artifacts_uri
from analysis.iclr.plotting import settings, fig_size, cb_line_cycler, cb_line_cycler_solid, cb_marker_cycler, savefig
from plot_hdf5_dataset import plot_dataset
from iclr_paper_path import get_paper_path
import matplotlib
settings()
import matplotlib.pyplot as plt

# beta 0.01
# path_dataset = filepath_from_ml_artifacts_uri('mlflow-artifacts:/829789035554334690/9e712d90248a4475b3bd7f1ff88f727e/artifacts/dataset.hdf5')
# beta 0.1 step
# path_dataset = filepath_from_ml_artifacts_uri('mlflow-artifacts:/799837826878041253/2f043616847c44edabb374d9df9a8838/artifacts/dataset.hdf5')
# beta 0.1 rrocs
# path_dataset = filepath_from_ml_artifacts_uri('mlflow-artifacts:/829789035554334690/d4ab1a5174054e12a8767e9c46d4827a/artifacts/dataset.hdf5')
# beta 0.1 rrcos steam 
# path_dataset = filepath_from_ml_artifacts_uri('mlflow-artifacts:/765349530991563084/b0b0669ff51c4ee5b05f203dc1d827c6/artifacts/dataset.hdf5')
# beta 0.1 step steam
path_dataset = filepath_from_ml_artifacts_uri('mlflow-artifacts:/807883014196875487/bd5f2795d5b148f6aa7ac8e6c100a78a/artifacts/dataset.hdf5')
# beta 0.1 step steam wo reparam
path_dataset = filepath_from_ml_artifacts_uri('mlflow-artifacts:/807883014196875487/a328c8f3946e4c1c8dc024fdafaac62f/artifacts/dataset.hdf5')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
dataset = h5py.File(path_dataset, 'r')
print(dataset.keys())

<KeysViewHDF5 ['common_test', 'common_validation', 'controls_names', 'outputs_names', 'states_der_names', 'states_names', 'test', 'time', 'train', 'validation']>


In [28]:
dataset['common_test'].keys()

<KeysViewHDF5 ['controls', 'kl_lat_controls', 'kl_lat_controls_per_dim', 'kl_lat_parameters', 'kl_lat_state_0', 'kl_lat_state_0_per_dim', 'kl_lat_states', 'kl_lat_states_per_dim', 'kl_loss', 'lat_controls_logvar', 'lat_controls_mu', 'lat_dim_controls_populated', 'lat_dim_parameters_populated', 'lat_dim_state_0_populated', 'lat_dim_states_populated', 'lat_state_0_logvar', 'lat_state_0_mu', 'lat_states_logvar', 'lat_states_mu', 'loss', 'ode_calls_forward', 'outputs', 'outputs_hat', 'reconstruction_loss', 'reconstruction_loss_outputs', 'reconstruction_loss_state_0', 'reconstruction_loss_states', 'rmse_outputs', 'rmse_state_0', 'rmse_states', 'states', 'states_der', 'states_hat', 'time_odeint', 'time_outputs']>

In [29]:
dataset_type = 'common_test'
# kl_lat_state_0_per_dim = dataset[dataset_type]['kl_lat_state_0_per_dim'][:]
lat_states_mu = dataset[dataset_type]['lat_states_mu'][:]
lat_states_logvar = dataset[dataset_type]['lat_states_logvar'][:]

kl_lat_states = dataset[dataset_type]['kl_lat_states_per_dim'][:]
# states = dataset[dataset_type]['states'][:]
# states_hat = dataset[dataset_type]['states_hat'][:]
time = dataset['time'][:]

# convert logvar to sigma
lat_states_sigma = np.exp(lat_states_logvar / 2)

# get populated states
kl_mean = np.mean(kl_lat_states, axis=(0,2))
# get idx for sorting
kl_mean_idx = np.flip(np.argsort(kl_mean))
print(kl_mean_idx)
print(kl_mean[kl_mean_idx])

[  3  76  93  98  25  64  53  94  17  68 117 122  61  16 124  41  86  52
   1  24 107  73  37  22  65  96  15  81  11 114 104   9  29  30  90  28
 120  21  26  47  95  59  66  56  71  92  97  69  45  31  19  36   6  75
 102 119  14  80   2  27 103  34  46  62  23 112 118  48  89  63 116  87
  70  83  51  99  85  10  35  54   7 123 105  77 100 101  40   5  13  55
   8 115  67  60  20  57 110 106  88  79  43 111  44 126  18   4   0  91
  49  72 113 125  38  84  50 127  39  32  33  42 109  82  74  78 108  58
 121  12]
[4.16719103e+00 2.71470952e+00 1.33331835e+00 1.26402414e+00
 1.23483753e+00 2.20522463e-01 1.03192240e-01 9.13732052e-02
 8.58076140e-02 7.45784938e-02 6.44266084e-02 5.87768704e-02
 5.08395620e-02 4.16841172e-02 4.05642726e-02 3.73281762e-02
 3.72547545e-02 3.54990959e-02 3.47159430e-02 3.44356336e-02
 3.25095803e-02 3.11879441e-02 2.98601389e-02 2.77619753e-02
 2.74558291e-02 2.48021781e-02 2.47976705e-02 2.24278420e-02
 2.14963723e-02 2.07474269e-02 2.07460672e-02 2.0536

In [30]:
n_first_states = 5
sample_idx = 5

fig, ax = plt.subplots(n_first_states,1 , sharex=True, figsize=fig_size(1.2, n_figs_per_width=3)) 
plt.subplots_adjust(wspace=0.1, hspace=0.1)

# set marker cycler
# ax.set_prop_cycle(cb_marker_cycler)

# kl_mean_idx[n_most_populated]

for ax_idx in range(0,n_first_states):
    ax[ax_idx].set_prop_cycle(cb_line_cycler_solid)
    idx = kl_mean_idx[ax_idx]
    ax[ax_idx].plot(time, lat_states_mu[sample_idx,idx, :], label=r'$\mu(\mathbf{x}^z_{'+str(idx)+ r'}(t))$', linewidth=0.2, color='black')
    ax[ax_idx].fill_between(time,
                            lat_states_mu[sample_idx,idx, :] - lat_states_sigma[sample_idx,idx, :], 
                            lat_states_mu[sample_idx,idx, :] + lat_states_sigma[sample_idx,idx, :], alpha=0.3, edgecolor='none')
    # ax[ax_idx].set_ylabel(r'$\mathbf{x}^z_{'+str(idx)+'}(t)$')
    # ax[ax_idx].set_ylim([-2, 2])
    # ax[ax_idx].legend(frameon=False, loc='upper right', fontsize=6)
    # ax[ax_idx].set_title(r'$\mathbf{x}^z_{'+str(idx)+'}(t)$', fontsize=5, pad = 1.5)
    # ax[ax_idx].set_ylim([-2,2])
ax[-1].set_xlabel('Time [s]')
# ax[-1].set_xlim(min(time), 1.5)
ax[0].set_title(r'$\mathbf{\mu}(\mathbf{x}^z_i(t))\pm \mathbf{\sigma}(\mathbf{x}^z_i(t))$', fontsize=8)


savefig(fig, get_paper_path('BNODE_steam', 'mu_sigma_traj'), dpi=600)
    

finding path that contains "code", then going up one level to find "LaTeX"
e:\projects\balanced_neural_odes_internal\code\python\analysis\iclr
e:\projects\balanced_neural_odes_internal\code\python\analysis
e:\projects\balanced_neural_odes_internal\code\python
e:\projects\balanced_neural_odes_internal\code
e:\projects\balanced_neural_odes_internal
E:\projects\balanced_neural_odes_internal\LaTeX\ICLR 2025 Template\figures\BNODE_steam\mu_sigma_traj.png
E:\projects\balanced_neural_odes_internal\LaTeX\ICLR 2025 Template\figures\BNODE_steam\mu_sigma_traj.svg


In [7]:
n_first_states = 5
sample_idx = 5

fig, ax = plt.subplots(2,n_first_states , sharex=True, figsize=fig_size(3, n_figs_per_width=1)) 
plt.subplots_adjust(wspace=0.1, hspace=0.5)

# set marker cycler
# ax.set_prop_cycle(cb_marker_cycler)

# kl_mean_idx[n_most_populated]

for ax_idx in range(0,n_first_states):
    ax[0,ax_idx].set_prop_cycle(cb_line_cycler_solid)
    idx = kl_mean_idx[ax_idx]
    ax[0,ax_idx].plot(time, lat_states_mu[sample_idx, idx, :], label=r'$\mu(\mathbf{x}^z_{'+str(idx)+ r'}(t))$', linewidth=0.2, color='black')
    ax[1,ax_idx].plot(time, lat_states_sigma[sample_idx, idx, :], label=r'$\sigma(\mathbf{x}^z_{'+str(idx)+ r'}(t))$', linewidth=0.2, color='black')
    # ax[ax_idx].set_ylabel(r'$\mathbf{x}^z_{'+str(idx)+'}(t)$')
    ax[0,ax_idx].set_ylim([-4, 4])
    # ax[ax_idx].legend(frameon=False, loc='upper right', fontsize=6)
    # ax[0,ax_idx].set_title(r'$\mathbf{x}^z_{'+str(idx)+'}(t)$', fontsize=6, pad = 3)
    # ax[0,ax_idx].set_ylim([-2,2])
# ax[-1].set_xlabel('Time [s]')
# ax[-1].set_xlim(min(time), 1.5)


# savefig(fig, get_paper_path('BNODE', 'mu_sigma_traj'), dpi=600)
    